In [1]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic

# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [2]:
def enforce_fundamental(F):
    U, S, VT = np.linalg.svd(F, full_matrices=False)
    if np.linalg.det(U @ VT) < 0:
        VT = -VT
    S[-1] = 0
    F = U @ np.diag(S) @ VT
    return F  

In [3]:
def estimate_F_DLT(img_pts_1, img_pts_2, print_svd=False): # Computes F such that x2.T @ F @ x1 = 0

    n = np.size(img_pts_1,1)
    M = []

    for i in range(n):

        # x1 = img_pts_1[0,i]
        # y1 = img_pts_1[1,i]
        # z1 = img_pts_1[2,i]
        
        # x2 = img_pts_2[0,i]
        # y2 = img_pts_2[1,i]
        # z2 = img_pts_2[2,i]

        # m = np.array([[x2*x1, x2*y1, x2*z1, y2*x1, y2*y1, y2*z1, z2*x1, z2*y1, z2*z1]])

        x = img_pts_1[:,i]
        y = img_pts_2[:,i]
        m = np.outer(y, x).flatten()
        M.append([m])


    M = np.concatenate(M, 0)
    U, S, VT = np.linalg.svd(M, full_matrices=False)
    F = VT[-1,:].reshape(3,3)
    F = enforce_fundamental(F)

    if print_svd:
        print('\nDet(F):', np.linalg.det(F))
        for i in range(np.size(img_pts_1,1)):
            epi_const = img_pts_2[:,i].T @ F @ img_pts_1[:,i]
            print('x2^T @ F @ x1:', epi_const)

        M_approx = U @ np.diag(S) @ VT
        v = VT[-1,:] # last row of VT because optimal v should be last column of V
        Mv = M @ v
        print('||Mv||:', (Mv @ Mv)**0.5)
        print('||v||^2:', v @ v)
        print('max{||M - M_approx||}:', np.max(np.abs(M - M_approx)))
        print('S:', S)

    return F

In [4]:
def unnormalise_F(N2, F_norm, N1):
    F = N2.T @ F_norm @ N1
    return F

In [5]:
def compute_epipolar_lines(F, x1, x2):
    l2 = F @ x1
    l1 = F.T @ x2
    return l1, l2

In [6]:
def compute_and_plot_lines(l, img, ax):

    col = cm.rainbow(np.linspace(0, 1, np.size(l,1)))

    for i in range(np.size(l,1)):

        a = l[0,i]
        b = l[1,i]
        c = l[2,i]

        x = np.linspace(0, np.size(img,1), 2)
        y = (-a*x - c) / b # ax + by + c = 0 ==> y = (-ax - c) / b

        ax.plot(x, y, '-', lw=3, color=col[i], alpha=0.7) #  label='Line {}'.format(i+1)

In [7]:
def plot_lines_points_and_image(img_pts, img, l, path, plt_img=False):

    fig = plt.figure(figsize=(8,6))
    ax = plt.axes()
    
    compute_and_plot_lines(l, img, ax)
    ax.plot(img_pts[0], img_pts[1], 'o', color='blue', label='Random points')

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    # plt.gca().invert_yaxis()
    # ax.invert_yaxis()
    ax.set_aspect('equal')
    ax.legend(loc="upper right")
    fig.tight_layout()

    if plt_img:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # ax.view_init(elev=-20, azim=-120, roll=60)
    fig.savefig(path, dpi=300)
    plt.show()


In [8]:
def point_line_distance_2D(l, p):

    D = []
    for i in range(np.size(l,1)):

        a = l[0,i]
        b = l[1,i]
        c = l[2,i]

        x = p[0,i]
        y = p[1,i]

        d = np.abs(a*x + b*y + c) / (a**2 + b**2)**0.5
        D.append([d])   
        
    D = np.concatenate(D, 0)
    return D

In [9]:
def compute_epipolar_errors(F, x1, x2):

    l1, l2 = compute_epipolar_lines(F, x1, x2)
    
    D1 = point_line_distance_2D(l1, x1)
    D2 = point_line_distance_2D(l2, x2)
    D_tot = np.concatenate((D1,D2),0)

    return D1, D2, D_tot

In [10]:
def plot_histogram(data, path):
    fig = plt.figure()
    plt.hist(data, bins=100, color='tab:blue')
    plt.xlabel('Error')
    plt.ylabel('Frequency')
    plt.xlim([0,4])
    fig.tight_layout()
    fig.savefig(path, dpi=300)
    plt.show()

In [12]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-3'
data = r'\A3data\data'
compex = r'\compEx1data.mat'
img1 = r'\kronan1.jpg'
img2 = r'\kronan2.jpg'
report = r'\report-images'

kronan1 = cv.load_image(path+data+img1)
kronan2 = cv.load_image(path+data+img2)
x = cv.convert_mat_to_np(path+data+compex, 'x')
x1 = cv.dehomogenize(x[0,0])
x2 = cv.dehomogenize(x[1,0])

norm = True
plt_pts = False
plt_hist = False

if norm:
    x1_norm, N1 = cv.normalise_x_and_y(x1)
    x2_norm, N2 = cv.normalise_x_and_y(x2)
else:
    N1 = N2 = np.eye(3)
    x1_norm = x1
    x2_norm = x2

print('\nx_mean:', np.mean(x1_norm[0,:]), '\nx_std:', np.std(x1_norm[0,:]), '\ny_mean:', np.mean(x1_norm[1,:]), '\ny_std:', np.std(x1_norm[1,:]))
print('\nx_mean:', np.mean(x2_norm[0,:]), '\nx_std:', np.std(x2_norm[0,:]), '\ny_mean:', np.mean(x2_norm[1,:]), '\ny_std:', np.std(x2_norm[1,:]))                                                             

F = estimate_F_DLT(x1_norm, x2_norm, print_svd=True)
F = unnormalise_F(N2, F, N1)
print('F:', F/F[-1,-1])
# x1 = cv.transform_and_dehomogenize(np.linalg.inv(N1), x1)
# x2 = cv.transform_and_dehomogenize(np.linalg.inv(N2), x2)

np.random.seed(0)
rand_mask = np.random.choice(np.size(x2,1), 20, replace=False)
x1_rand = x1[:,rand_mask]
x2_rand = x2[:,rand_mask]
l1, l2 = compute_epipolar_lines(F, x1_rand, x2_rand)

if norm:
    path1 = path+report+'/CE1_kronan1_norm.png'
    path2 = path+report+'/CE1_kronan2_norm.png'
else:
    path1 = path+report+'/CE1_kronan1_unnorm.png'
    path2 = path+report+'/CE1_kronan2_unnorm.png'

if plt_pts:
    plot_lines_points_and_image(x1_rand, kronan1, l1, path1, plt_img=True)
    plot_lines_points_and_image(x2_rand, kronan2, l2, path2, plt_img=True)

D1, D2, D_tot = compute_epipolar_errors(F, x1, x2)

if norm:
    print('\nMean distance norm 1:', np.mean(D1))
    print('Mean distance norm 2:', np.mean(D2))
    print('Mean distance norm tot:', np.mean(D_tot))
else:
    print('\nMean distance unnorm 1:', np.mean(D1))
    print('Mean distance unnorm 2:', np.mean(D2))
    print('Mean distance unnorm tot:', np.mean(D_tot))

if norm:
    path1 = path+report+'/CE1_hist_norm_1.png'
    path2 = path+report+'/CE1_hist_norm_2.png'
    path3 = path+report+'/CE1_hist_norm_tot.png'
else:
    path1 = path+report+'/CE1_hist_unnorm_1.png'
    path2 = path+report+'/CE1_hist_unnorm_2.png'
    path3 = path+report+'/CE1_hist_unnorm_tot.png'

if plt_hist:
    plot_histogram(D1, path1)
    plot_histogram(D2, path2)
    plot_histogram(D_tot, path3)


x_mean: 851.2593625498008 
x_std: 420.16769192483406 
y_mean: 667.675876494024 
y_std: 332.98670637196153

x_mean: -7.077118882072711e-18 
x_std: 1.0 
y_mean: -2.618533986366903e-16 
y_std: 1.0

x_mean: 760.4540239043824 
x_std: 454.03769079569935 
y_mean: 664.1767231075696 
y_std: 346.1775864736366

x_mean: 1.5569661540559965e-16 
x_std: 1.0 
y_mean: 5.378610350375261e-16 
y_std: 1.0000000000000002

x_mean: -7.077118882072711e-18 
x_std: 1.0 
y_mean: -2.618533986366903e-16 
y_std: 1.0

x_mean: 1.5569661540559965e-16 
x_std: 1.0 
y_mean: 5.378610350375261e-16 
y_std: 1.0000000000000002

Det(F): 2.330211244168243e-18
x2^T @ F @ x1: 2.153306781843556e-05
x2^T @ F @ x1: 0.0015210735099888961
x2^T @ F @ x1: 0.0007368044857557488
x2^T @ F @ x1: -0.0005452851603851006
x2^T @ F @ x1: 0.0010096556260803524
x2^T @ F @ x1: -9.139998095597512e-05
x2^T @ F @ x1: 0.0015541594223791866
x2^T @ F @ x1: -0.001630549721091723
x2^T @ F @ x1: -0.001232725023418757
x2^T @ F @ x1: -0.0013904159190061827
x2